# Code Associated with Research Question 1

__Research Question 1__ <br>
"Is there a change in the low-frequency power between _not learned_ and _learned_ conditions?"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import neuropsy as npsy
import neuropsy.analysis as npsya
import pickle
import json
import pandas
import time
from scipy import signal
from scipy.fftpack import next_fast_len

### Decide Center Frequencies and Number of Cycles for Complex Morelet Wavelet

In [ ]:
def plot_wavelets(frequencies, cycles, t_wl, fs):
    
    nrows = np.floor(np.sqrt(len(frequencies))).astype(int)
    
    fig, ax = plt.subplots(figsize=(25, nrows*4), nrows=nrows, ncols=4)#, sharex=True, sharey=True)
    fig2, ax2 = plt.subplots(figsize=(25, nrows*4), nrows=nrows, ncols=4)#, sharex=True, sharey=True)
    ax = ax.flatten()
    ax2 = ax2.flatten()

    wavelet_funcs = {}
    for i, (f, n) in enumerate(zip(frequencies, cycles)):

        # create complex Morlet wavelet
        wavelet, gauss_env = npsya.morlet(f, n, t_wl, return_gaussian=True)
        wavelet_funcs[f] = wavelet
        
        # plot wavelet and gaussian envelope
        ax[i].plot(t_wl, gauss_env, ':', label='envelope', color='black')
        ax[i].plot(t_wl, wavelet.real, label='real')
        ax[i].plot(t_wl, wavelet.imag, label='imaginary', linestyle='--', linewidth=1.)
        ax[i].set_title(f'f = {f}, cycles = {n}')
        if (i % 4 == 0) or (i % 8 == 0) or (i % 12 == 0) or (i % 16 == 0):
            ax[i].set_ylabel('Amplitude')
        if i == 0:
            ax[-1].set_xlabel('Time (s)')
            ax[-2].set_xlabel('Time (s)')
            ax[-3].set_xlabel('Time (s)')
            ax[-4].set_xlabel('Time (s)')
        ax[i].legend(loc='upper right', shadow=True)
        
        # plot frequency spectrum
        xf, y = npsya.compute_fft(wavelet, fs, output='mag')
        ax2[i].plot(xf, y)
        ax2[i].set_title(f'f = {f}, cycles = {n}')
        if (i % 4 == 0) or (i % 8 == 0) or (i % 12 == 0) or (i % 16 == 0):
            ax2[i].set_ylabel('Magnitude')
        if i == 0:
            ax2[-1].set_xlabel('Frequency (Hz)')
            ax2[-2].set_xlabel('Frequency (Hz)')
            ax2[-3].set_xlabel('Frequency (Hz)')
            ax2[-4].set_xlabel('Frequency (Hz)')
        # ax2[i, j].set_yscale('log')
        ax2[i].set_xlim([-1, 40])
        # ax2[i, j].set_ylim([40, 120])
            
    fig.suptitle('Complex Morlet wavelet - Time Domain', fontsize=16, y=.92)
    fig2.suptitle('Complex Morlet wavelet - Frequency Domain', fontsize=16, y=.92)
    plt.show()
    
    
# ********* TEST FUNCTION ********* #
fs                          = 512
t_wl                        = np.arange(-4, 4+1/fs, 1/fs) # long enough time to capture the wavelet for given f and n

# OLD
# frequencies                 = np.arange(2, 32, 2)
# cycles                      = np.insert(np.flipud(np.arange(3, 13, 1)), [-1, -2, -3, -4, -5], [3, 4, 5, 6, 7])

# CURRENT
# frequencies                 = np.arange(2, 13, 1)
# cycles                      = [8, 7, 7, 6, 6, 5, 5, 4, 4, 3, 3]

# TEST
frequencies                 = np.arange(2, 13, 1)
# cycles                      = [8, 7, 7, 6, 6, 5, 5, 4, 4, 3, 3]
# cycles = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]

plot_wavelets(frequencies, cycles, t_wl, fs)

# 1. Time-frequency Power Analysis 

- Analyse Entire hippocampus
- Manual convolution
- 4 conditions ("not learned", "correct", "learned-1", "learned") 
- Time bins between stimulus onset (picture shown) and stimulus offset (picture placed)
- Baseline correct within trials (-1 s to -0.6 s before stimulus onset)


- NaNs are removed during preprocessing
- outliers are removed here before power analysis

Steps:
1. Iterate over all subjects
2. Compute time-frequency power for the channels in each subject
3. Cut trial epochs and corresponding baseline period
4. Baseline correct trial epochs
5. Average time-frequency power across trials to obtain channel average per condition
6. Average time-frequency power across subjects and across channels to obtain condition average
7. Average all time-frequency power across all conditions to obtain total average

In [2]:
#********** PARAMETERS **********#
path_data                   = '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/data/preprocessed'
path_save                   = f'/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis'
subject_ids                 = npsy.utils.get_subject_ids_from_path(path_data)

# analysis parameters
conditions                  = ('not learned', 'correct', 'learned-1', 'learned')
cond_column                 = 'Condition'
stimulus_onset              = 'Picture Shown'
stimulus_offset             = 'Picture Placed'
nbins_between               = 512
save_postfix                = 'picshownandplaced'
tmin                        = -.5                   # time before stimulus onset
tmax                        = 1                     # time after stimulus offset
baseline                    = (-1, -.5)             # time period from before stimulus onset to use for baseline correction
baseline_method             = 'mean'                # basline correction method (see )

# wavelet parameters
fs                          = 512
t_wl                        = np.arange(-4, 4+1/fs, 1/fs) # long enough time to capture the wavelet for given f and n
# [INFO] BEFORE (2-30 Hz, 3-12 cycles)
# frequencies                 = np.arange(2, 32, 2)
# cycles                      = np.insert(np.flipud(np.arange(3, 13, 1)), [-1, -2, -3, -4, -5], [3, 4, 5, 6, 7])
# [INFO] AFTER (2-12 Hz, 3-8 cycles)
frequencies                 = np.arange(2, 13, 1)
# cycles                      = np.insert(np.flipud(np.arange(3, 9, 1)), [-1, -2, -3, -4, -5], [3, 4, 5, 6, 7])
cycles                      = [8, 7, 6, 6, 5, 5, 5, 4, 4, 3, 3]
n_half_wavelet              = len(t_wl) // 2

# plot parameters
cmap                        = 'RdBu_r'

# set up directory for saving results
if not os.path.exists(path_save):
    os.makedirs(path_save)
    os.makedirs(f"{path_save}/data")

### 1.1 Power Analysis

Code computes fft of a channel within a subject once, then iterates over conditions and trials.

In [ ]:
# save general analysis info (will be saved as json file in the end)
dict_analysis_info = {}
dict_analysis_info['general'] = {}
dict_analysis_info['general']['conditions'] = conditions
dict_analysis_info['general']['cond_column'] = cond_column
dict_analysis_info['general']['stimulus_onset'] = stimulus_onset
dict_analysis_info['general']['stimulus_offset'] = stimulus_offset
dict_analysis_info['general']['nbins_between'] = nbins_between
dict_analysis_info['general']['tmin'] = tmin
dict_analysis_info['general']['tmax'] = tmax
dict_analysis_info['general']['baseline'] = baseline
dict_analysis_info['general']['baseline_method'] = baseline_method
dict_analysis_info['general']['wavelet'] = {'frequencies': frequencies.tolist(), 'cycles': cycles.tolist() if isinstance(cycles, np.ndarray) else cycles}

# time-frequency result dictionary {condition: {subject_id: {channel: {trial: np.2darray}}}}
dict_tfr_power_results = {c: {s: {} for s in subject_ids} for c in conditions}

#********** START ANALYSIS **********#
for subject_id in subject_ids:
    
    # initialise variables
    trial_names_outliers                = None
    labels_outliers                     = None
    dict_outliers                       = {}
    trial_names_too_short_or_close      = None
    labels_too_short_or_close           = None
    dict_drop                           = {}
    
    
    start_time_sub = time.time()
    print(f"Starting subject {subject_id}...")

    #********** LOAD SUBJECT DATA **********# 
    data = npsy.DataHandler(path=path_data, subject_id=subject_id, exp_phase=2, fs=512, verbose=False)
    data.load(load_saved=True, postfix='preprocessed')
    
    # subtract total mean from each channel to remove DC bias
    print("Removing DC bias from iEEG data...")
    total_mean = np.mean(data.ieeg)
    for i in range(data.ieeg.shape[0]):
        data.ieeg[i, :] = data.ieeg[i, :] - total_mean
        data.ieeg[i, :] = signal.detrend(data.ieeg[i, :])
        
    # select channels in hippocampus
    print(f"Selecting channels in hippocampus...")
    ch_names = data.df_chan.loc[data.df_chan['DK_Subfields'].str.contains('HP_head|HP_body|HP_tail', case=True)].name.to_list()
    # check that there are channels in the hippocampal subfield for this subject, otherwise skip
    if len(ch_names) > 0:
        data.select_channels(ch_names=np.ravel(ch_names), inplace=True)
        print(f"Selected {len(ch_names)} channels in hippocampus.")
    else:
        print(f"No channels in hippocampus.")
    
    # remove outliers
    print("Removing outliers from experiment dataframe...")
    idx_outliers = data.df_exp[data.df_exp['outlier'] == True]['outlier'].index.to_list()
    if len(idx_outliers) > 0:
        trial_names_outliers    = data.df_exp[data.df_exp['outlier'] == True]['Trial Identifier'].to_list()
        labels_outliers         = data.df_exp.loc[idx_outliers, cond_column].to_list()
        for i, label in zip(idx_outliers, labels_outliers):
            if label not in dict_outliers.keys():
                dict_outliers[label] = 1
            else:
                dict_outliers[label] += 1
        for key, value in dict_outliers.items():
            print(f"{value} outliers in condition {repr(key)}.")
        data.df_exp = data.df_exp.drop(idx_outliers).reset_index(drop=True)
    else:
        print("No outliers in experiment dataframe.")

    # [INFO] need to make sure trials have enough time points for the chosen nbins_between
    #********** CHECK TRIAL DURATIONS **********# 
    print("Checking trial durations...")
    print(f"\tmax duration between onset and offset is {round(nbins_between / fs, 2)} seconds.")
    idx_too_short, idx_too_close = npsya.check_trials(df=data.df_exp, col_name=cond_column, tmin=tmin, tmax=round(nbins_between / fs, 2), baseline=baseline)
    idx_drop = np.unique(np.concatenate((idx_too_short, idx_too_close))).astype(int).tolist()
    if len(idx_drop) > 0:
        trial_names_too_short_or_close  = data.df_exp.loc[idx_drop, 'Trial Identifier'].to_list()
        labels_too_short_or_close       = data.df_exp.loc[idx_drop, cond_column].to_list()
        for i, label in zip(idx_drop, labels_too_short_or_close):
            if label not in dict_drop.keys():
                dict_drop[label] = 1
            else:
                dict_drop[label] += 1
        for key, value in dict_drop.items():
            print(f"{value} are too short or too close with other trials in condition {repr(key)}.")
        data.df_exp = data.df_exp.drop(idx_drop).reset_index(drop=True)
    else:
        print("No trials are too short or too close with other trials.")

    #********** CREATE MNE RAW OBJECT WITH iEEG DATA **********#
    data.create_raw()
    

    #********** GET TRIAL TIME POINTS (INDICES) AND TRIAL IDENTIFIERS (NAMES) **********#
    # - need trial indices for conditions from experiment dataframe
    # - also need the corresponding trial indices for the baseline (picture shown)
    # - also need to extract the trial identifiers (names) to keep track of trials in the resulting time-frequency power data
    dict_trial_indices = {**{f'{c}_onset': None for c in conditions}, **{f'{c}_offset': None for c in conditions}, **{f'{c}_baseline': None for c in conditions}, **{f'{c}_names': None for c in conditions}}
    for c in conditions:
        # get the trial indices for the condition
        idx_onset                               = data.df_exp[data.df_exp[cond_column] == c][f'Mark for {stimulus_onset}'].to_numpy().astype(int)
        idx_offset                              = data.df_exp[data.df_exp[cond_column] == c][f'Mark for {stimulus_offset}'].to_numpy().astype(int)
        idx_baseline                            = data.df_exp[data.df_exp[cond_column] == c]['Mark for Picture Shown'].to_numpy().astype(int)
        # save indices
        dict_trial_indices[f"{c}_onset"]        = idx_onset
        dict_trial_indices[f"{c}_offset"]       = idx_offset
        dict_trial_indices[f"{c}_baseline"]     = idx_baseline
        # get the trial identifiers (names) for the condition
        trial_names                             = data.df_exp[data.df_exp[cond_column] == c]['Trial Identifier'].to_numpy().astype(str)
        # save trial identifiers
        dict_trial_indices[f"{c}_names"]        = trial_names


    #********** TIME-FREQUENCY ANALYSIS **********#
    if len(ch_names) > 0:
        
        # CHANNEL:
        #  - FFT of entire channel signal
        #  - CWT for each frequency and cycle
        #  - store power in dB for each frequency
        #  - this is done once per channel, then afterwards each trial is considered independently during CONDITION loop
        #    - this is done to save time as the CWT is the most time-consuming part
        for ch in ch_names:
            start_time_ch = time.time()
            print(f"\tStarting channel {ch}...")
            
            # use the whole channel signal for computing time-frequency representation
            ch_signal = data.raw._data[ch_names.index(ch), ...]
            
            # ********* FFT of CHANNEL ********* #
            n_conv      = len(t_wl) + len(ch_signal) - 1
            n_conv_fast = next_fast_len(n_conv)
            signal_fft  = np.fft.fft(ch_signal, n_conv_fast)
            
            # initialise output data for continuous wavelet transform
            tf_data = np.zeros((len(frequencies), len(ch_signal)))
            
            # ********* CWT ********* #
            for i, (f, n) in enumerate(zip(frequencies, cycles)):
                
                # [INFO] - print current wavelet parameters
                # print(f"creating wavelet with parameters: f = {f}, n = {n}...")
                
                # create wavelet
                wavelet = npsya.morlet(f, n, t_wl)
                
                # fft of wavelet
                # note:
                #   output must match the length of the signal_fft in order to multiply in the frequency domain
                wavelet_fft = np.fft.fft(wavelet, n=n_conv_fast)
                
                # convolution
                coefficients = np.fft.ifft(signal_fft * wavelet_fft, n=n_conv_fast)
                coefficients = coefficients[:n_conv] # remove padding from next_fast_len
                coefficients = coefficients[n_half_wavelet:-n_half_wavelet]
                
                # convert to power in dB
                tf_power = 10 * np.log10(np.abs(coefficients)**2)
                
                # store result for frequency
                tf_data[i, :] = tf_power
                
            # clean up
            del ch_signal, signal_fft, wavelet, wavelet_fft, coefficients, tf_power
                
            # CONDITION:
            #  - continuous wavelet transform result is stored in tf_data (in dB)
            #  - now we need to consider each trial independently
            #  - loop over conditions
            #  - loop over trials in condition
            #  - baseline correction is done for each trial separately
            #  - the window between onset and offset is divided into bins
            for condition in conditions:
                start_time_cond = time.time()
                print(f"\t\tStarting condition {repr(condition)}...")
                
                dict_tfr_power_results[condition][subject_id][ch] = {}
                
                for (i_onset, i_offset, i_baseline, trial_id) in zip(dict_trial_indices[f'{condition}_onset'], 
                                                                     dict_trial_indices[f'{condition}_offset'], 
                                                                     dict_trial_indices[f'{condition}_baseline'], 
                                                                     dict_trial_indices[f'{condition}_names']):
                    start_time_trial = time.time()
                    
                    # get power for trial period
                    i_tmin = int(i_onset + int(tmin * fs))
                    i_tmax = int(i_offset + int(tmax * fs))
                    power_trial = tf_data[:, i_tmin:i_tmax]
                    # get power for baseline period
                    b_tmin = int(i_baseline + int(baseline[0] * fs))
                    b_tmax = int(i_baseline + int(baseline[1] * fs))
                    power_baseline = tf_data[:, b_tmin:b_tmax]
                    # baseline correction
                    tf_data[:, i_tmin:i_tmax] = npsya.apply_baseline(epoch_power=power_trial, baseline_power=power_baseline, method=baseline_method)
                    
                    # create bins between onset and offset
                    power_bins = np.array_split(tf_data[:, i_onset:i_offset], nbins_between, axis=1)
                    # average bins across time (this will be the new time points, only this time in bins)
                    power_bins_avg = np.zeros((len(frequencies), nbins_between))
                    for i, power_bin in enumerate(power_bins):
                        power_bins_avg[:, i] = np.mean(power_bin, axis=1)
                    
                    # store time-frequency power result for trial
                    dict_tfr_power_results[condition][subject_id][ch][trial_id] = np.concatenate([tf_data[:, i_tmin:i_onset], power_bins_avg, tf_data[:, i_offset:i_tmax]], axis=1)
                    
                    print(f"\t\t\tTrial {trial_id} done in {(time.time() - start_time_trial)*1000:.2f} milliseconds")
                    
                # clean up
                del power_trial, power_baseline, power_bins, power_bins_avg
                
                print(f"\t\tCondition {repr(condition)} done - {time.time() - start_time_cond:.2f} seconds")
            print(f"\tChannel {ch} done - {time.time() - start_time_ch:.2f} seconds")
    # no channels
    else:
        # remove subject from results if there are no channels in the hippocampus
        for condition in conditions:
            dict_tfr_power_results[condition].pop(subject_id)
        print(f"No channels in hippocampus for subject {subject_id}.")
        

    # save analysis info for subject
    tmp_dict_conditions         = {f'{c}': len(dict_trial_indices[f'{c}_onset']) for c in conditions}
    tmp_dict_outliers           = {**{f'{c}': dict_outliers.get(c, 0) for c in conditions}, **{'count': len(idx_outliers)}, **{'indices': idx_outliers}, **{'trial_identifiers': trial_names_outliers}}
    tmp_dict_too_short_or_close = {**{f'{c}': dict_drop.get(c, 0) for c in conditions}, **{'count': len(idx_drop)}, **{'indices': idx_drop}, **{'trial_identifiers': trial_names_too_short_or_close}}
    
    dict_analysis_info[f'subject {subject_id}'] = {
        'trials_kept': tmp_dict_conditions if len(ch_names) > 0 else None,
        'trials_dropped': {
            'outliers': tmp_dict_outliers,
            'too_short_or_too_close': tmp_dict_too_short_or_close
        } if len(ch_names) > 0 else None,
        'channels': ch_names if len(ch_names) > 0 else None
    }
    
    print(f"Subject {subject_id} done - {time.time() - start_time_sub:.2f} seconds")
    
    # break
        

#********** SAVE RESULTS **********#
filename = f"{path_save}/data/tfr_power_results.pkl"
with open(filename, 'wb') as f:
    print(f"Saving results as {repr(filename)}...")
    pickle.dump(dict_tfr_power_results, f)
    print("Done")
#********** SAVE ANALYSIS INFO **********#
filename = f"{path_save}/data/tfr_analysis_info.json"
with open(filename, 'w') as f:
    print(f"Saving analysis info as {repr(filename)}...")
    json.dump(dict_analysis_info, f)
    print("Done")

### 1.2 Load Time-Frequency Power Results

In [3]:
# [INFO] load data in this cell once to avoid re-running the heavy loading process. 
filename = f"{path_save}/data/tfr_power_results.pkl"

# load previously computed time-frequency power for all subjects
with open(filename, 'rb') as f:
    dict_tfr_power_results = pickle.load(f)

# get conditions and subject ids from the loaded results dictionary
conditions = list(dict_tfr_power_results.keys())
subject_ids = list(dict_tfr_power_results[conditions[0]].keys())

# compute channel average (average each channel across all trials)
dict_tfr_power_results_chan_avg = {c: {s: {} for s in subject_ids} for c in conditions}
for condition in conditions:
    for subject_id in subject_ids:
        for ch in dict_tfr_power_results[condition][subject_id].keys():
            dict_tfr_power_results_chan_avg[condition][subject_id][ch] = np.mean(list(dict_tfr_power_results[condition][subject_id][ch].values()), axis=0)
            
# compute subject average (average each subject across all channels)
dict_tfr_power_results_subj_avg = {c: {s: {} for s in subject_ids} for c in conditions}
for condition in conditions:
    for subject_id in subject_ids:
        dict_tfr_power_results_subj_avg[condition][subject_id] = np.mean(list(dict_tfr_power_results_chan_avg[condition][subject_id].values()), axis=0)
        
# compute condition average (average each condition across all subjects)
dict_tfr_power_results_cond_avg = {c: {} for c in conditions}
for condition in conditions:
    dict_tfr_power_results_cond_avg[condition] = np.mean(list(dict_tfr_power_results_subj_avg[condition].values()), axis=0)
    
# clean up to save memory
del dict_tfr_power_results

### 1.3 Visualise Time-Frequency Representation - Channel Average

plot each channel separately for each subject, plot shows all conditions separately and the average across trials.

In [6]:
# individual color bars (True/False)
individ_cbars = False

for subject_id in subject_ids:
    for ch in list(dict_tfr_power_results_chan_avg[conditions[0]][subject_id].keys()):
        
        fig, ax = plt.subplots(ncols=len(conditions), nrows=1, figsize=(15, 3), layout='compressed')
        
        for i, condition in enumerate(conditions):
            
            im = ax[i].imshow(
                dict_tfr_power_results_chan_avg[condition][subject_id][ch],
                aspect='auto',
                extent=[0, 1279, (frequencies)[0], (frequencies)[-1]],
                origin='lower', 
                vmax=np.max(data) if individ_cbars else 3,
                vmin=-np.max(data) if individ_cbars else -3,
                cmap=cmap,
                interpolation='hanning')
            ax[i].set_title(f'{condition} - {ch}', fontsize=12)
            ax[0].set_ylabel('Frequency (Hz)', fontsize=11) # only set ylabel for left most plots
            # ax[i].set_xlabel('Trial Period', fontsize=11)
            ax[i].set_yticks(frequencies, frequencies)
            
            # add line at onset and offset
            ax[i].axvline(x=255, color='black', linestyle='--', linewidth=1.)
            ax[i].axvline(x=255+512, color='black', linestyle='--', linewidth=1.)
            
            # set custom x ticks to account for bins and time points being plotted together
            xticks  = [0, 127, 255, 355, 455, 555, 655, 767, 1023, 1279]
            xlabels = ["-0.5 s", "-0.25 s", 'onset', 'b100', 'b200', 'b300', 'b400', 'offset', "0.5 s", "1 s"]
            ax[i].set_xticks(xticks, xlabels, rotation=45, fontsize=10)
            
            if individ_cbars:
                # add colorbar for each plot separately
                cbar = plt.colorbar(im, ax=ax[i], shrink=0.70, pad=0.01, orientation="vertical")
                if i == len(conditions)-1: cbar.ax.set_ylabel("dB difference", rotation=270, labelpad=15, fontsize=11)
        
        if not individ_cbars:
            # add colorbar for the entire figure
            cbar = plt.colorbar(im, ax=ax.ravel().tolist(), shrink=0.70, pad=0.01, orientation="vertical")
            cbar.ax.set_ylabel("dB from baseline", rotation=270, labelpad=15, fontsize=11)
        fig.suptitle(f"Subject {subject_id} - Hippocampus", fontsize=14, y=1.06, x=.49)
        
        full_save_path = f"{path_save}/1.3 sub{subject_id}_{ch}_power.png"
        fig.savefig(full_save_path, dpi=300, bbox_inches='tight')
        print(f"Saved figure for subject {subject_id} at {repr(full_save_path)}")
        plt.close(fig)
        # plt.show()
            
    # break

Saved figure for subject 03 at "/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.3 sub03_B' 02_power.png"
Saved figure for subject 03 at "/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.3 sub03_B' 03_power.png"
Saved figure for subject 03 at "/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.3 sub03_B' 04_power.png"
Saved figure for subject 03 at "/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.3 sub03_B' 05_power.png"
Saved figure for subject 03 at "/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.3 sub03_C' 02_power.png"
Saved figure for subject 04 at '/mnt/c/Users/matti/OneDrive/Educa

### 1.4 Visualise Time-Frequency Representation - Subject Average

In [7]:
# individual color bars (True/False)
individ_cbars = False

#********** PLOT RESULTS **********#
for subject_id in subject_ids:
    
    fig, ax = plt.subplots(ncols=len(conditions), nrows=1, figsize=(15, 3), layout='compressed')
    
    for i, condition in enumerate(conditions):
            
        im = ax[i].imshow(
            dict_tfr_power_results_subj_avg[condition][subject_id], 
            aspect='auto',
            extent=[0, 1279, (frequencies)[0], (frequencies)[-1]],
            origin='lower', 
            vmin=-np.max(dict_tfr_power_results_subj_avg[condition][subject_id]) if individ_cbars else -2,
            vmax=np.max(dict_tfr_power_results_subj_avg[condition][subject_id]) if individ_cbars else 2,
            cmap=cmap,
            interpolation='hanning')
        
        ax[i].set_title(f'{condition}', fontsize=12)
        ax[0].set_ylabel('Frequency (Hz)', fontsize=11)        # only set ylabel for left most plots
        # ax[i].set_xlabel('Trial Period', fontsize=11)
        ax[i].set_yticks(frequencies, frequencies)
        
        # add line at onset and offset
        ax[i].axvline(x=255, color='black', linestyle='--', linewidth=1.)
        ax[i].axvline(x=255+512, color='black', linestyle='--', linewidth=1.)
        
        # set custom x ticks to account for bins and time points being plotted together
        xticks  = [0, 127, 255, 355, 455, 555, 655, 767, 1023, 1279]
        xlabels = ["-0.5 s", "-0.25 s", 'onset', 'b100', 'b200', 'b300', 'b400', 'offset', "0.5 s", "1 s"]
        ax[i].set_xticks(xticks, xlabels, rotation=45, fontsize=10)
        
        if individ_cbars:
            # add colorbar for the entire figure
            cbar = plt.colorbar(im, ax=ax[i], shrink=0.70, pad=0.01, orientation="vertical")
            cbar.ax.set_ylabel("dB difference", rotation=270, labelpad=15, fontsize=11)

    if not individ_cbars:
        # add colorbar for the entire figure
        cbar = plt.colorbar(im, ax=ax.ravel().tolist(), shrink=0.70, pad=0.01, orientation="vertical")
        cbar.ax.set_ylabel("dB difference", rotation=270, labelpad=15, fontsize=11)
    fig.suptitle('Hippocampus Average', fontsize=14, y=1.06, x=.49)

    full_save_path = f"{path_save}/1.4 sub{subject_id}_power.png"
    fig.savefig(full_save_path, dpi=300, bbox_inches='tight')
    print(f"Saved figure for subject {subject_id} at {repr(full_save_path)}")
    plt.close(fig)

Saved figure for subject 03 at '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.4 sub03_power.png'
Saved figure for subject 04 at '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.4 sub04_power.png'
Saved figure for subject 05 at '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.4 sub05_power.png'
Saved figure for subject 07 at '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.4 sub07_power.png'
Saved figure for subject 09 at '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.4 sub09_power.png'
Saved figure for subject 10 at '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-p

### 1.5 Visualise Time-Frequency Representation - Condition Average

In [8]:
# individual color bars (True/False)
individ_cbars = False

#********** PLOT RESULTS **********#
fig, ax = plt.subplots(ncols=len(conditions), nrows=1, figsize=(15, 3), layout='compressed')
for i, condition in enumerate(conditions):
        
    im = ax[i].imshow(
        dict_tfr_power_results_cond_avg[condition], 
        aspect='auto',
        extent=[0, 1279, (frequencies)[0], (frequencies)[-1]],
        origin='lower', 
        vmin=-np.max(dict_tfr_power_results_cond_avg[condition]) if individ_cbars else -2,
        vmax=np.max(dict_tfr_power_results_cond_avg[condition]) if individ_cbars else 2,
        cmap=cmap,
        interpolation='hanning')
    
    ax[i].set_title(f'{condition}', fontsize=12)
    ax[0].set_ylabel('Frequency (Hz)', fontsize=11)        # only set ylabel for left most plots
    # ax[i].set_xlabel('Trial Period', fontsize=11)
    ax[i].set_yticks(frequencies, frequencies)
    
    # add line at onset and offset
    ax[i].axvline(x=255, color='black', linestyle='--', linewidth=1.)
    ax[i].axvline(x=255+512, color='black', linestyle='--', linewidth=1.)
    
    # set custom x ticks to account for bins and time points being plotted together
    xticks  = [0, 127, 255, 355, 455, 555, 655, 767, 1023, 1279]
    xlabels = ["-0.5 s", "-0.25 s", 'onset', 'b100', 'b200', 'b300', 'b400', 'offset', "0.5 s", "1 s"]
    ax[i].set_xticks(xticks, xlabels, rotation=45, fontsize=10)
    
    if individ_cbars:
        # add colorbar for the entire figure
        cbar = plt.colorbar(im, ax=ax[i], shrink=0.70, pad=0.01, orientation="vertical")
        cbar.ax.set_ylabel("dB difference", rotation=270, labelpad=15, fontsize=11)

if not individ_cbars:
    # add colorbar for the entire figure
    cbar = plt.colorbar(im, ax=ax.ravel().tolist(), shrink=0.70, pad=0.01, orientation="vertical")
    cbar.ax.set_ylabel("dB difference", rotation=270, labelpad=15, fontsize=11)
fig.suptitle('Hippocampus Average', fontsize=14, y=1.06, x=.49)

full_save_path = f"{path_save}/1.5 condition_power.png"
fig.savefig(full_save_path, dpi=300, bbox_inches='tight')
print(f"Saved figure at {repr(full_save_path)}")
plt.close(fig)

Saved figure at '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/Research Question 1/Time-frequency Power Analysis/1.5 condition_power.png'
